In [1]:
import sys
import numpy as np
from scipy.integrate import odeint
import os
from tqdm import tqdm  # Import tqdm for progress bar

# Pendulum rod lengths (m), bob masses (kg).
L1, L2 = 1, 1
m1, m2 = 1, 1
# The gravitational acceleration (m.s-2).
g = 9.81
print("starting...")

def deriv(y, t, L1, L2, m1, m2):
    """Return the first derivatives of y = theta1, z1, theta2, z2."""
    theta1, z1, theta2, z2 = y
    c, s = np.cos(theta1-theta2), np.sin(theta1-theta2)
    theta1dot = z1
    z1dot = (m2*g*np.sin(theta2)*c - m2*s*(L1*z1**2*c + L2*z2**2) -
             (m1+m2)*g*np.sin(theta1)) / L1 / (m1 + m2*s**2)
    theta2dot = z2
    z2dot = ((m1+m2)*(L1*z1**2*s - g*np.sin(theta2) + g*np.sin(theta1)*c) + 
             m2*L2*z2**2*s*c) / L2 / (m1 + m2*s**2)
    return theta1dot, z1dot, theta2dot, z2dot

def calc_E(y):
    """Return the total energy of the system."""
    th1, th1d, th2, th2d = y.T
    V = -(m1+m2)*L1*g*np.cos(th1) - m2*L2*g*np.cos(th2)
    T = 0.5*m1*(L1*th1d)**2 + 0.5*m2*((L1*th1d)**2 + (L2*th2d)**2 +
            2*L1*L2*th1d*th2d*np.cos(th1-th2))
    return T + V

# Maximum time, time point spacings, and the time grid (all in s).
tmax, dt = 360000, 0.001
t = np.arange(0, tmax+dt, dt)
# Initial conditions: theta1, dtheta1/dt, theta2, dtheta2/dt.
y0 = np.array([3*np.pi/7, 0, 3*np.pi/4, 0])

# Break the time array into chunks to use with tqdm
chunk_size = 1000
chunks = [t[i:i + chunk_size] for i in range(0, len(t), chunk_size)]

# Define the file path
file_path = f"pendulum_data{tmax}.npz"

# Initialize storage lists
time_data, theta1_data, theta2_data = [], [], []
x1_data, y1_data, x2_data, y2_data, energy_data = [], [], [], [], []

# Use tqdm to show the progress
for chunk in tqdm(chunks, desc="Integrating", unit="chunk"):
    # Perform the numerical integration on each chunk
    y_chunk = odeint(deriv, y0, chunk, args=(L1, L2, m1, m2))

    # Unpack theta and convert to Cartesian coordinates.
    theta1, theta2 = y_chunk[:, 0], y_chunk[:, 2]
    x1 = L1 * np.sin(theta1)
    y1 = -L1 * np.cos(theta1)
    x2 = x1 + L2 * np.sin(theta2)
    y2 = y1 - L2 * np.cos(theta2)

    # Calculate the energy
    E = calc_E(y_chunk)

    # Store data
    time_data.append(chunk)
    theta1_data.append(theta1)
    theta2_data.append(theta2)
    x1_data.append(x1)
    y1_data.append(y1)
    x2_data.append(x2)
    y2_data.append(y2)
    energy_data.append(E)

    # Update initial conditions for the next chunk
    y0 = y_chunk[-1]  # The last value of this chunk is the initial condition for the next chunk

# Convert lists to NumPy arrays
np.savez(file_path,
         Time=np.concatenate(time_data),
         Theta1=np.concatenate(theta1_data),
         Theta2=np.concatenate(theta2_data),
         X1=np.concatenate(x1_data),
         Y1=np.concatenate(y1_data),
         X2=np.concatenate(x2_data),
         Y2=np.concatenate(y2_data),
         Energy=np.concatenate(energy_data))

print(f"Data saved to {file_path}")

# Load and print the saved data (optional)
loaded_data = np.load(file_path)
print("Loaded Data:")
print({key: loaded_data[key].shape for key in loaded_data.keys()})

# Convert NPZ data to a Pandas DataFrame
df = pd.DataFrame({key: data[key] for key in data.keys()})

# Print the first few rows
print(tabulate(df, headers='keys', tablefmt='psql'))

starting...


Integrating: 100%|██████████| 360001/360001 [23:25<00:00, 256.21chunk/s]


Data saved to pendulum_data360000.npz
Loaded Data:
{'Time': (360000001,), 'Theta1': (360000001,), 'Theta2': (360000001,), 'X1': (360000001,), 'Y1': (360000001,), 'X2': (360000001,), 'Y2': (360000001,), 'Energy': (360000001,)}


In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate


# Load the NPZ file
file_path = "pendulum_data30000.npz"  # Ensure this matches your saved file
data = np.load(file_path)

print(data.files)  # List all the keys in the NPZ file
for key in data.files:
    print(f"{key}: {data[key].shape}")  # Check the shape of each array

# Convert NPZ data to a Pandas DataFrame
df = pd.DataFrame({key: data[key] for key in data.keys()})

print(df.head())  # This is faster and gives a quick preview
print(df.shape)  # Prints number of rows and columns in the DataFrame

# Print the first few rows
print(tabulate(df.head(), headers='keys', tablefmt='psql'))  # Only print first 10 rows

print(tabulate(df.tail(), headers='keys', tablefmt='psql'))  # Only print first 10 rows


['Time', 'Theta1', 'Theta2', 'X1', 'Y1', 'X2', 'Y2', 'Energy']
Time: (30000001,)
Theta1: (30000001,)
Theta2: (30000001,)
X1: (30000001,)
Y1: (30000001,)
X2: (30000001,)
Y2: (30000001,)
Energy: (30000001,)
    Time    Theta1    Theta2        X1        Y1        X2        Y2    Energy
0  0.000  1.346397  2.356194  0.974928 -0.222521  1.682035  0.484586  2.570857
1  0.001  1.346392  2.356193  0.974927 -0.222525  1.682034  0.484581  2.570857
2  0.002  1.346379  2.356190  0.974924 -0.222538  1.682034  0.484565  2.570857
3  0.003  1.346356  2.356185  0.974919 -0.222560  1.682033  0.484540  2.570857
4  0.004  1.346325  2.356177  0.974912 -0.222591  1.682031  0.484504  2.570857
(30000001, 8)
+----+--------+----------+----------+----------+-----------+---------+----------+----------+
|    |   Time |   Theta1 |   Theta2 |       X1 |        Y1 |      X2 |       Y2 |   Energy |
|----+--------+----------+----------+----------+-----------+---------+----------+----------|
|  0 |  0     |  1.3464  |  

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data from .npz file
print("Loading data...")
data = np.load(file_path)
data_key = list(data.keys())[0]  # Assuming data is stored under the first key
df_cpu = data[data_key]
print("Data loaded successfully!")

# Standardization on CPU
print("Standardizing data...")
mean = np.mean(df_cpu, axis=0)
std = np.std(df_cpu, axis=0)
scaled_data_cpu = (df_cpu - mean) / std
print("Data standardized!")

# Convert to PyTorch tensor and move to GPU if available
print("Moving data to GPU (if available)...")
scaled_data_gpu = torch.tensor(scaled_data_cpu, dtype=torch.float32, device=device)
print("Data moved to", device)

# GPU-accelerated KMeans (PyTorch implementation)
def kmeans_torch(X, n_clusters, max_iter=100):
    print(f"Running KMeans clustering with {n_clusters} clusters...")
    n_samples, n_features = X.shape
    centroids = X[torch.randperm(n_samples)[:n_clusters]]

    for i in range(max_iter):
        distances = torch.cdist(X, centroids)
        labels = distances.argmin(dim=1)
        new_centroids = torch.stack([X[labels == i].mean(dim=0) for i in range(n_clusters)])
        
        if torch.all(centroids == new_centroids):
            print(f"KMeans converged at iteration {i + 1}")
            break
        centroids = new_centroids
    
    print("KMeans clustering completed!")
    return labels, centroids

# Run KMeans
clusters, _ = kmeans_torch(scaled_data_gpu, n_clusters=3)

# GPU-accelerated PCA (using torch.pca_lowrank)
print("Performing PCA for dimensionality reduction...")
U, S, V = torch.pca_lowrank(scaled_data_gpu, q=2)
reduced_data_gpu = scaled_data_gpu @ V[:, :2]
print("PCA completed!")

# Convert back to CPU for visualization
print("Converting PCA results to CPU...")
reduced_data_np = reduced_data_gpu.cpu().numpy()
clusters_np = clusters.cpu().numpy()
print("Conversion complete!")

# Scatter plot of PCA results
print("Plotting PCA results...")
plt.scatter(reduced_data_np[:, 0], reduced_data_np[:, 1], c=clusters_np, cmap="viridis", alpha=0.7)
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("PyTorch-Accelerated PCA Projection of Clusters")
plt.colorbar(label="Cluster")
plt.show()
print("PCA plot generated!")

print("All tasks completed successfully!")


Using device: cuda
Loading data...
Data loaded successfully!
Standardizing data...
Data standardized!
Moving data to GPU (if available)...
Data moved to cuda
Running KMeans clustering with 3 clusters...


ValueError: not enough values to unpack (expected 2, got 1)